# Cosim

In [ ]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [ ]:
ams.__version__

In [ ]:
andes.config_logger(stream_level=20)

In [ ]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

In [ ]:
ams.config_logger(stream_level=10)

In [ ]:
sp = ams.load(ams.get_case('ieee14/ieee14.xlsx'),
              setup=True,)

sa = sp.to_andes(overwrite=True,
                 addfile=andes.get_case('ieee14/ieee14_dyn_only.xlsx'),
                 no_output=True,)

In [ ]:
sa.PFlow.run()

In [ ]:
sa.TDS.run()

In [ ]:
sp.ACOPF.pg

In [ ]:
sp.DCOPF.run()

In [ ]:
sp.ACOPF.run()

In [ ]:
sp.ACOPF.pg.get_idx()

In [ ]:
sp.ACOPF.pg

In [ ]:
sp.sync_andes(sa)